# Statistical Inference

## Setup

In [1]:
# Load libraries
library(ggplot2)
library(ggpubr)
library(fitdistrplus)
library(nortest)
library(car)

Loading required package: MASS

Loading required package: survival

Loading required package: carData



In [2]:
# Load dataset
data <- read.csv("../results/cardiovascular_disease_clean.csv", 
                colClasses = c("numeric", "factor", "integer", "numeric", "integer", "integer", "factor", "factor", "factor", "factor", "factor", "factor", "numeric", "numeric"),
                col.names=c("Age(years)", "Gender", "Height(cm)", "Weight(kg)", "SystolicPressure", "DiastolicPressure", "Cholesterol", "Glucose", "Smoke", "Alcohol", "Active", "CardiovascularDisease", "BMI", "Pulse"))
numeric.cols <- c(1, 3, 4, 5, 6, 13, 14)
paste("The dataset has", dim(data)[2], "features for", dim(data)[1], "patients.")

[1] "The dataset has 14 features for 67240 patients."

## Check data distributions

In [3]:
# Function to get kernel density plots
plot_density <- function(column){
    ggplot(data, aes(x=data[,column], fill=CardiovascularDisease)) + 
    scale_fill_manual(values=c(No="cyan3", Yes="salmon")) +
    geom_density(alpha=0.5) +
    labs(x=column) +
    theme(axis.text=element_text(size=15), 
    axis.title.x = element_text(size = 20), 
    axis.title.y = element_text(size = 20),
    legend.title = element_text(size = 20), 
    legend.text = element_text(size = 20)) 
}

# Plot histogram for each numeric column
plots <- lapply(colnames(data[, numeric.cols]), plot_density)

# Save
plot.arrange <- ggarrange(plots[[1]],plots[[2]],plots[[3]],plots[[6]], plots[[4]],
          plots[[5]],plots[[7]], nrow=3, ncol=3)

ggsave("../docs/assests/numeric_cols_kde.png",bg = "white", plot = plot.arrange, width = 35, height = 15)


### Kernel density plots for numeric variables
![](../docs/assests/numeric_cols_kde.png)

The kernel density plots show that the numeric columns do not follow a normal distribution. Indeed, the blood pressure data do not seem to be continuous; instead, they appear to be discrete. For that reason, from now we will handle these variables as discrete. 

In [4]:
# Update numerical columns
numeric.cols <- c(1,3,4,13)

# Transform pressure variables in 10 sized intervals
data$SystolicPressure <- as.factor(round(data$SystolicPressure / 10) * 10)
data$DiastolicPressure <- as.factor(round(data$DiastolicPressure / 10) * 10)
data$Pulse <- as.factor(round(data$Pulse / 10) * 10)

plot_histogram <- function(column){
    ggplot(data, aes(x = data[,column], fill=CardiovascularDisease)) +
        scale_fill_manual(values=c(No="cyan3", Yes="salmon")) +
        geom_bar(position="dodge") +
        labs(x=column) +
        theme(axis.text=element_text(size=15), 
        axis.title.x = element_text(size = 20), 
        axis.title.y = element_text(size = 20),
        legend.title = element_text(size = 20), 
        legend.text = element_text(size = 20))
}

plots <- lapply(c("SystolicPressure", "DiastolicPressure", "Pulse"), plot_histogram)
plot.arrange <- ggarrange(plots[[1]],plots[[2]],plots[[3]], nrow=1, ncol=3)
ggsave("../docs/assests/categorical_cols_hist.png",bg = "white", plot = plot.arrange, width = 30, height = 10)


### Histograms for categorical pressure variables
![](../docs/assests/categorical_cols_hist.png)

Interestingly, the divergence between the plots of healthy and diseased patients in both the numerical and categorical variables, suggests that there might be a statistical association between these variables and cardiovascular disease. In the following sections, we will test these hypotheses.

## Normality tests

For each numeric variable, we present a graphical representation of its distribution, the results of the Kolmogorov-Smirnov normality test, as well as the results of the Fligner-Killeen test of homogeneity of variances. Given that we empirically observed differences between healthy and diseased patients, we decided to test for normality in each group separately.

In [5]:
# Get Cullen and Frey graphs
plot_descdist <- function(colname, cardio="Yes"){
    png(paste("../docs/assests/", colname, "_cardio_", cardio,"_dists.png", sep=""), bg = "white", width = 900, height = 700)
    descdist(data[data$CardiovascularDisease == cardio, colname])
    dev.off()
}

lapply(colnames(data[, numeric.cols]), plot_descdist, cardio="Yes")
lapply(colnames(data[, numeric.cols]), plot_descdist, cardio="No")

[[1]]
pdf 
  2 

[[2]]
pdf 
  2 

[[3]]
pdf 
  2 

[[4]]
pdf 
  2

[[1]]
pdf 
  2 

[[2]]
pdf 
  2 

[[3]]
pdf 
  2 

[[4]]
pdf 
  2

### Cullen and Frey graph for Age(years)
 <div style="display: flex; justify-content: space-around; align-items: center">
  <figure style="text-align: center;">
    <img src="../docs/assests/Age.years._cardio_No_dists.png" alt="Healthy" width="100%">
    <figcaption>Healthy</figcaption>
  </figure>
  <figure style="text-align: center;">
    <img src="../docs/assests/Age.years._cardio_Yes_dists.png" alt="Cadiovascular disease" width="100%">
    <figcaption>Cardiovascular disease</figcaption>
  </figure>
</div>

In [6]:
# Kolmogorov-Smirnov normality test
lillie.test(data[data$CardiovascularDisease == "No", "Age.years."])
lillie.test(data[data$CardiovascularDisease == "Yes", "Age.years."])


	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "No", "Age.years."]
D = 0.059064, p-value < 2.2e-16



	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "Yes", "Age.years."]
D = 0.071113, p-value < 2.2e-16


In [7]:
# Fligner-Killeen test of homogeneity of variances
fligner.test(data$Age.years., data$CardiovascularDisease)


	Fligner-Killeen test of homogeneity of variances

data:  data$Age.years. and data$CardiovascularDisease
Fligner-Killeen:med chi-squared = 171.97, df = 1, p-value < 2.2e-16


### Cullen and Frey graph for Height(cm)
 <div style="display: flex; justify-content: space-around; align-items: center">
  <figure style="text-align: center;">
    <img src="../docs/assests/Height.cm._cardio_No_dists.png" alt="Healthy" width="100%">
    <figcaption>Healthy</figcaption>
  </figure>
  <figure style="text-align: center;">
    <img src="../docs/assests/Height.cm._cardio_Yes_dists.png" alt="Cadiovascular disease" width="100%">
    <figcaption>Cardiovascular disease</figcaption>
  </figure>
</div>

In [8]:
# Kolmogorov-Smirnov normality test
lillie.test(data[data$CardiovascularDisease == "No", "Height.cm."])
lillie.test(data[data$CardiovascularDisease == "Yes", "Height.cm."])


	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "No", "Height.cm."]
D = 0.05194, p-value < 2.2e-16



	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "Yes", "Height.cm."]
D = 0.049813, p-value < 2.2e-16


In [9]:
# Fligner-Killeen test of homogeneity of variances
fligner.test(data$Height.cm., data$CardiovascularDisease)


	Fligner-Killeen test of homogeneity of variances

data:  data$Height.cm. and data$CardiovascularDisease
Fligner-Killeen:med chi-squared = 26.126, df = 1, p-value = 3.199e-07


### Cullen and Frey graph for Weight(kg)
 <div style="display: flex; justify-content: space-around; align-items: center">
  <figure style="text-align: center;">
    <img src="../docs/assests/Weight.kg._cardio_No_dists.png" alt="Healthy" width="100%">
    <figcaption>Healthy</figcaption>
  </figure>
  <figure style="text-align: center;">
    <img src="../docs/assests/Weight.kg._cardio_Yes_dists.png" alt="Cadiovascular disease" width="100%">
    <figcaption>Cardiovascular disease</figcaption>
  </figure>
</div>

In [10]:
# Kolmogorov-Smirnov normality test
lillie.test(data[data$CardiovascularDisease == "No", "Weight.kg."])
lillie.test(data[data$CardiovascularDisease == "Yes", "Weight.kg."])


	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "No", "Weight.kg."]
D = 0.083471, p-value < 2.2e-16



	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "Yes", "Weight.kg."]
D = 0.070426, p-value < 2.2e-16


In [11]:
# Fligner-Killeen test of homogeneity of variances
fligner.test(data$Weight.kg., data$CardiovascularDisease)


	Fligner-Killeen test of homogeneity of variances

data:  data$Weight.kg. and data$CardiovascularDisease
Fligner-Killeen:med chi-squared = 322.56, df = 1, p-value < 2.2e-16


### Cullen and Frey graph for BMI
 <div style="display: flex; justify-content: space-around; align-items: center">
  <figure style="text-align: center;">
    <img src="../docs/assests/BMI_cardio_No_dists.png" alt="Healthy" width="100%">
    <figcaption>Healthy</figcaption>
  </figure>
  <figure style="text-align: center;">
    <img src="../docs/assests/BMI_cardio_Yes_dists.png" alt="Cadiovascular disease" width="100%">
    <figcaption>Cardiovascular disease</figcaption>
  </figure>
</div>

In [12]:
# Kolmogorov-Smirnov normality test
lillie.test(data[data$CardiovascularDisease == "No", "BMI"])
lillie.test(data[data$CardiovascularDisease == "Yes", "BMI"])


	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "No", "BMI"]
D = 0.08471, p-value < 2.2e-16



	Lilliefors (Kolmogorov-Smirnov) normality test

data:  data[data$CardiovascularDisease == "Yes", "BMI"]
D = 0.070455, p-value < 2.2e-16


In [13]:
# Fligner-Killeen test of homogeneity of variances
fligner.test(data$BMI, data$CardiovascularDisease)


	Fligner-Killeen test of homogeneity of variances

data:  data$BMI and data$CardiovascularDisease
Fligner-Killeen:med chi-squared = 561.39, df = 1, p-value < 2.2e-16


The results of the Kolmogorov-Smirnov test confirm that none of our numerical variables follow a normal distribution. In addition, the Fligner-Killeen test shows that there are differences in the variances of the variables between the healthy and diseased patients. Therefore, to test for the statistical differences in the median, we will employ the non-paramtric Wilcoxon Rank Sum Test.

## Statistical test for numerical variables

To test the hypotheses that the patients with cardiovascular disease have greater age, weight and BMI than the healthy patients we used the Wilcoxon Rank Sum Test with alternative = "greater". 

In [14]:
# Function to get boxplots
get_boxplot <- function(column){
    wilcoxon = wilcox.test(x=data[data$CardiovascularDisease == "Yes", column],
                           y=data[data$CardiovascularDisease == "No", column],
                           alternative = "greater")
    ggplot(data, aes(x = CardiovascularDisease, y=data[,column], fill=CardiovascularDisease)) +
     geom_violin(color=NA) +
     geom_boxplot(fill="white", width=0.1) +
     labs(x="Cardiovascular Condition", y=column) +
    theme(axis.text=element_text(size=15), 
        axis.title.x = element_text(size = 20), 
        axis.title.y = element_text(size = 20),
        legend.title = element_text(size = 20), 
        legend.text = element_text(size = 20)) +
    geom_text(x = 0.9, y = max(data[data$CardiovascularDisease == "No", column]), 
              label = paste("p-value =", round(wilcoxon$p.value, 4)), color = "black", size = 10) +
    scale_fill_manual(values = c(Yes="salmon", No="cyan3"))
}

# Plot histogram for each numeric column
plots <- lapply(colnames(data[, numeric.cols]), get_boxplot)

# Save
plot.arrange <- ggarrange(plots[[1]],plots[[2]],plots[[3]],plots[[4]], nrow=2, ncol=2)

ggsave("../docs/assests/numeric_cols_boxplots.png",bg = "white", plot = plot.arrange, width = 30, height = 20)


### Distribution comparisons for numeric variables
![](../docs/assests/numeric_cols_boxplots.png)

The p-values displayed in the boxplots are rounded to four decimals. For the variables Age, Weight and BMI, the p-values are so small that thery are virtually 0. Thus, the differences observed in the medians are statistically significant. 

## Statistical tests for categorical variables

Besides the numerical variables, the dataset contains 9 catagorical features: Gender. Cholesterol, Glucose, Smoke, Alcohol, Active, and the three blood pressure related variables discussed above, Systolic Pressure, Disatolic Pressure and Pulse.
To test if each of these categorical variables is independent of the cardiovascular condition of the patient, we apply the Chi-square test.

In [15]:
get_barplot <- function(column){
    chi.sq = chisq.test(x = data[, column], y=data$CardiovascularDisease)
    ggplot(data, aes(x = CardiovascularDisease,y = after_stat(count), fill=data[,column])) +
     geom_bar() + labs(fill=column) +
     theme(axis.text=element_text(size=15), 
        axis.title.x = element_text(size = 20), 
        axis.title.y = element_text(size = 20),
        legend.title = element_text(size = 20), 
        legend.text = element_text(size = 20)) +
        annotate("text", x = 0.9, y = 31000, label = paste("p-value =", round(chi.sq$p.value,4)), color = "black", size = 10) 
}


# Plot histogram for each numeric column
plots <- lapply(colnames(data[, c(-numeric.cols, -12)]), get_barplot)

# Save
plot.arrange <- ggarrange(plots[[2]],plots[[3]],plots[[9]],
          plots[[1]],plots[[4]],plots[[5]],
          plots[[6]],plots[[7]],plots[[8]], nrow=3, ncol=3)

ggsave("../docs/assests/categorical_cols_barplots.png",bg = "white", plot = plot.arrange, width = 30, height = 20)

Warning message in chisq.test(x = data[, column], y = data$CardiovascularDisease):
“Chi-squared approximation may be incorrect”
Warning message in chisq.test(x = data[, column], y = data$CardiovascularDisease):
“Chi-squared approximation may be incorrect”


### Categorical variables comparisons
![](../docs/assests/categorical_cols_barplots.png)

The plot shows that all the variables but the Gender have a statistically significant relationship with a patient's health status. Therfore, all this variables, alongside the numerical variables, may be a valuable source of information to train a model to predict cardiovascular disease. 